# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [33]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [34]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hobyo,5.3505,48.5268,78.89,83,70,10.78,SO,1706488060
1,1,port-aux-francais,-49.3500,70.2167,40.33,70,91,30.96,TF,1706488020
2,2,sisimiut,66.9395,-53.6735,-6.29,98,100,9.69,GL,1706488054
3,3,iqaluit,63.7506,-68.5145,-33.07,61,75,12.66,CA,1706487751
4,4,puerto natales,-51.7236,-72.4875,53.15,78,99,4.72,CL,1706488028


### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [37]:
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles = 'OSM', 
    frame_width = 700, 
    frame_height = 500, 
    color='City',
    alpha= 0.5,
    s='Humidity',
    scale= 0.8)


# Display the map
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [38]:
# Define criteria for ideal weather condition
ideal_weather_condition = (
    (city_data_df['Max Temp'] < 27) & 
    (city_data_df['Max Temp'] > 21)&
)
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[ideal_weather_condition].dropna()

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
59,59,cherepovets,59.1333,37.9000,25.79,96,100,8.08,RU,1706488297
67,67,khowy,38.5503,44.9521,24.37,74,82,3.44,IR,1706488299
75,75,olonkinbyen,70.9221,-8.7187,25.84,72,100,20.02,SJ,1706488300
176,176,lyozna,55.0249,30.7956,26.17,96,100,5.28,BY,1706488314
188,188,mil'kovo,55.6078,37.7956,25.29,99,89,5.26,RU,1706488077


### Step 3: Create a new DataFrame called `hotel_df`.

In [39]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df = hotel_df.assign(Hotel_Name='')

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel_Name
0,hobyo,SO,5.3505,48.5268,83,
1,port-aux-francais,TF,-49.3500,70.2167,70,
2,sisimiut,GL,66.9395,-53.6735,98,
3,iqaluit,CA,63.7506,-68.5145,61,
4,puerto natales,CL,-51.7236,-72.4875,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [42]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accommodation.hotel',\
         'filter':'',\
         'bias':'',\
         'apiKey':geoapify_key,\
         'limit':1}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
#         # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
#     # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# # Display sample data
hotel_df.head()

Starting hotel search
hobyo - nearest hotel: Iftin Hotel Hobyo
port-aux-francais - nearest hotel: Keravel
sisimiut - nearest hotel: Sømandshjemmet
iqaluit - nearest hotel: Frobisher Inn
puerto natales - nearest hotel: Dorotea Patagonia Hostel
rio preto da eva - nearest hotel: Hotel Santo Antonio
yellowknife - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
happy valley-goose bay - nearest hotel: Hotel North Two
port elizabeth - nearest hotel: Waterford Hotel
tiksi - nearest hotel: Арктика
santa vitoria do palmar - nearest hotel: Hotel Brasil
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
blackmans bay - nearest hotel: Villa Howden
krasnogvardeyskoye - nearest hotel: No hotel found
hami - nearest hotel: Хами
dwarka - nearest hotel: The Dwarika Hotel
edinburgh of the seven seas - nearest hotel: No hotel found
vilyuchinsk - nearest hotel: Победа
tarawa - nearest hotel: Betio Lodge
coquimbo - nearest hotel: Hotel Iberia
hawaiian paradise park - nearest 

santaluz - nearest hotel: No hotel found
tafresh - nearest hotel: هتل پردیسان
yalta - nearest hotel: Таврида
bandon - nearest hotel: The Munster Arms Hotel
piti village - nearest hotel: The Cliff Hotel
kuhdasht - nearest hotel: هتل ازادی
gereida - nearest hotel: No hotel found
ghadamis - nearest hotel: فندق عين الفارس
manaure - nearest hotel: hotel palaima
kalmunai - nearest hotel: Saji new food
pisco - nearest hotel: La Portada
palmer - nearest hotel: No hotel found
keflavik - nearest hotel: Núpan Deluxe
retreat - nearest hotel: No hotel found
tabuk - nearest hotel: Davidson Hotel
kidal - nearest hotel: No hotel found
ilhabela - nearest hotel: Vila Kebaya
tabou - nearest hotel: hôtel le rochet
jamestown - nearest hotel: DoubleTree Jamestown
clearlake - nearest hotel: Best Western El Grande Inn
acapulco de juarez - nearest hotel: Hotel del Valle
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
gadzhiyevo - nearest hotel: Оазис
mikuni - nearest hotel: 三国観光ホテル
san patricio - neare

chonchi - nearest hotel: Hotel Huildín
la'ie - nearest hotel: No hotel found
yarmouth - nearest hotel: The Inn At Cape Cod
tarnogskiy gorodok - nearest hotel: Русь
hasaki - nearest hotel: 大新旅館
port said - nearest hotel: The New Continental Hotel
ghat - nearest hotel: فندق تاسيلي
dryden - nearest hotel: Holiday Inn Express
mermoz boabab - nearest hotel: La maison blanche
luanda - nearest hotel: Hotel Forum
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)
ouallam - nearest hotel: No hotel found
tromso - nearest hotel: Clarion Collection Hotel Aurora
qandala - nearest hotel: No hotel found
brisas de zicatela - nearest hotel: Casa de Olas
karachi - nearest hotel: Quetta Agha Hotel
merizo village - nearest hotel: No hotel found
villa maria - nearest hotel: Gran Hotel Orión
zunyi - nearest hotel: 香山宾馆
kiunga - nearest hotel: No hotel found
betio village - nearest hotel: Seaman Hostel
wasagu - nearest hotel: No hotel found
maun - nearest hotel: Center Lodge Conference Ce

urucui - nearest hotel: Formula Flat hotel
cole harbour - nearest hotel: Hearthstone Inn Boutique Hotel Dartmouth/Halifax
huarmey - nearest hotel: Hostal Santa Rosa
kalabo - nearest hotel: Golden Lodge Luxury Acomodation
shanhecun - nearest hotel: No hotel found
central point - nearest hotel: La Quinta Inn & Suites by Wyndham Central Point - Medford
ulaangom - nearest hotel: АНДУУД
layou - nearest hotel: High Field Guesthouse


,City,Country,Lat,Lng,Humidity,Hotel_Name,Hotel Name
0,hobyo,SO,5.3505,48.5268,83,,Iftin Hotel Hobyo
1,port-aux-francais,TF,-49.3500,70.2167,70,,Keravel
2,sisimiut,GL,66.9395,-53.6735,98,,Sømandshjemmet
3,iqaluit,CA,63.7506,-68.5145,61,,Frobisher Inn
4,puerto natales,CL,-51.7236,-72.4875,78,,Dorotea Patagonia Hostel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [46]:
# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles='OSM', 
    frame_width=700, 
    frame_height=500, 
    color='City',
    alpha=0.6,
    hover=True,
    hover_cols=['City', 'Country', 'Hotel Name', 'Humidity'],
    use_index=False,
    s='Humidity',
    scale=1.2
)

# Display the map
hotel_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)